In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import re

In [ ]:
import sys
sys.path.append('../..')

In [ ]:
import cmaps

In [ ]:
sns.set_style('white')
sns.set_color_codes()
sns.set_context('paper')

In [ ]:
colwidth=246 / 72

In [ ]:
eigvals2e = {}
for f in glob.glob('../outputs/eigval_2elec_*.csv'):
    m = re.search(r'eigval_2elec_om(\d*\.?\d+)\.csv', f)
    if m:
        name = float(m.group(1))
    eigvals2e[name] = pd.read_csv(f, names=(name,), header=0).values.ravel()

eigvecs2e = {}
for f in glob.glob('../outputs/eigvec_2elec_*.csv'):
    m = re.search(r'eigvec_2elec_om(\d*\.?\d+)\.csv', f)
    if m:
        name = float(m.group(1))
        eigvecs2e[name] = pd.read_csv(f, header=None).values
    else:
        raise ValueError('RE didn\'t work')
        
eigvecs1e = pd.read_csv('../outputs/eigvec_lowest3.csv', header=None).values
eigvals1e = pd.read_csv('../outputs/eigval_lowest3.csv', header=0).values.ravel()

In [ ]:
for i, v in eigvals2e.items():
    idx = v.argsort()
    eigvecs2e[i] = eigvecs2e[i][:, idx]

eigvecs1e = eigvecs1e[:, eigvals1e.argsort()]

In [ ]:
armavec = pd.read_csv('../outputs/arma_eigvec_lowest3.csv', header=None).values
armaval = pd.read_csv('../outputs/arma_eigval_lowest3.csv', header=None).values

In [ ]:
plt.imshow((eigvecs1e[:, :20]**2 - armavec[:, :20]**2).T, interpolation='nearest', cmap='RdBu')
plt.colorbar()
plt.gca().set_aspect('auto')

In [ ]:
np.sort(eigvals1e)[:50].ravel() - armaval[:50].ravel()

In [ ]:
plt.figure(figsize=(colwidth, colwidth/1.6))
plt.imshow((eigvecs1e[:, :20]**2).T, interpolation='nearest', origin='lower', cmap=cmaps.viridis,
           extent=(0, 10, 0, 20))
plt.colorbar().set_label(r'$|u|^2(\rho)$')
plt.gca().set_aspect('auto')
plt.xlabel(r'$\rho$')
plt.ylabel('Eigenvector number')
plt.savefig('eigenvecs.pdf', bbox_inches='tight')

In [ ]:
omega = 0.5

plt.imshow((eigvecs2e[omega][:, :10]**2).T, origin='lower', interpolation='nearest')
plt.gca().set_aspect('auto')

In [ ]:
plt.imshow((eigvecs1e[:, :10]**2).T, origin='lower', interpolation='nearest', extent=(0, 10, 0, 10),
           cmap='Greys')
plt.gca().set_aspect('auto')

In [ ]:
with sns.axes_style('white'):   
    pal = sns.color_palette('Set2')
    fig, ax = plt.subplots(len(eigvecs2e), 3, sharey='row', sharex=True, figsize=(6, 4.5))
    for i, omega in enumerate(sorted(eigvecs2e.keys())):
        for n in (0, 1, 2):
            plt.sca(ax[i][n])
            m = eigvecs2e[omega]
            xs1 = np.linspace(0, 10, eigvecs1e.shape[0])
            xs2 = np.linspace(0, 10, len(m))
            plt.plot(xs1, eigvecs1e[:, n]**2, 'k', label='One-electron case')
            plt.fill_between(xs2, m[:, n]**2, label=r'$\omega_r$ = {}'.format(omega), color=pal[i])
    #             plt.legend()
            sns.despine(left=True)
            plt.grid(axis='y')
            if n == 0:
                plt.ylabel(r'$u^2(\rho)$')
            if omega == max(eigvecs2e.keys()):
                plt.xlabel(r'$\rho$')
            if omega == min(eigvecs2e.keys()):
                plt.title('n = {}'.format(n))
            if n == 2:
                plt.text(1.03, 0.5, r'$\omega_r$ = {}'.format(omega), transform=plt.gca().transAxes, 
                         rotation='vertical', ha='left', va='center')
    plt.xlim(0, 10)

    plt.tight_layout()
    plt.savefig('wavefunctions.pdf')

In [ ]:
for i, v in eigvals2e.items():
    print(i, np.sort(v)[:5])

In [ ]:
1 / 5

In [ ]:
np.sort(eigvals2e[0.01])[:3] * 2

# Scaling

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
scal = pd.read_csv('../outputs/scaling_iters.csv')

In [ ]:
def fitfunc(x, a, b):
    return a*x**2 + b*x

In [ ]:
def powermodel(x, a, b):
    return a*x**b
def quadmodel(x, a, b, c):
    return a*x**2 + b*x + c
def cubemodel(x, a, b, c, d):
    return a*x**3 + b*x**2 + c*x + d

In [ ]:
plt.figure(figsize=(colwidth, colwidth/1.5))
plt.plot(scal.N, scal.iters, 'bo', label='Measurements')
xs = np.linspace(0, 300, 100)
fparam, fcov = curve_fit(quadmodel, scal.N, scal.iters)
print(fparam)
plt.plot(xs, quadmodel(xs, *fparam), 'b', label='Quadratic fit')
sns.despine()
# plt.loglog()
# plt.grid(axis='y')
plt.xlim(0, 310)
plt.ylim(-2000, 160000)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
plt.xlabel('Dimension of matrix')
plt.ylabel('Num. transformations')
plt.legend(loc='upper left')
plt.savefig('transformScaling.pdf', bbox_inches='tight')

In [ ]:
plt.plot(scal.N, scal.iters - quadmodel(scal.N, *fparam), 'o')

In [ ]:
plt.figure(figsize=(colwidth, colwidth/1.5))

d = scal[scal.N >= 100]
fp2, fc2 = curve_fit(powermodel, d.N.values, d.time.values)
print(fp2)
afp2, afc2 = curve_fit(powermodel, d.N.values, d.arma_time.values)
print(afp2)

xs = np.linspace(100, 300, 100)
plt.plot(d.N, d.time, 'bo', label='Jacobi data')
plt.plot(xs, powermodel(xs, *fp2), 'b-', label='Jacobi fit')
plt.plot(d.N, d.arma_time, 'rs', label='Arma. data')
plt.plot(xs, powermodel(xs, *afp2), 'r--', label='Arma. fit')
plt.loglog()
plt.xticks(np.arange(100, 310, 40), np.arange(100, 310, 40))

# sns.despine()
plt.grid(axis='both')

plt.xlim(90, 310)
plt.xlabel('Dimension of matrix')
plt.ylabel('Time [s]')
plt.legend(loc=2, frameon=True)

plt.savefig('timing.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, sharey=False, squeeze=True)

plt.sca(ax[0])
plt.stem(d.N, d.time - powermodel(d.N, *fp2), linefmt='b-', markerfmt='bo', basefmt='b-')
plt.xlim(90, 310)
plt.hlines(0, *plt.xlim(), color='b')
plt.ylim(-0.1, 0.1)
plt.ylabel('Jacobi residual')

plt.sca(ax[1])
plt.stem(d.N, d.arma_time - powermodel(d.N, *afp2), linefmt='r-', markerfmt='rs', basefmt='r-')
plt.hlines(0, *plt.xlim(), color='r')
plt.ylim(-0.007, 0.007)
plt.xlabel('Dimension of matrix')
plt.ylabel('Arma. residual')

# plt.tight_layout()

In [ ]:
fp2